In [166]:
'''All packages'''

import geopandas as gpd
import pandas as pd
from shapely import geometry
from shapely.geometry import Polygon, LineString, Point
import os
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt, mpld3
from mpld3 import fig_to_html, plugins
import random


In [167]:
#main variables 

directory = "./archives"
ext = ".geojson"

#variable that contains all the searched filters
lista=[]

listadirs=[]
dic={}
filtrolist=[]
fp={}

'''plt.rcParams['figure.figsize'] = (20, 10) #setting parameters'''
df_places = gpd.read_file('./Lisboa.geojson') #read file Lisboa.geojson
bug = gpd.read_file('./archives/economia/Feiras.geojson') #read file Lisboa.geojson

In [168]:
#function to search



def leitura(filtro = "NULL"):
    i=0
    sum=0
    global lista, listadirs, dic, filtrolist
    listafiles=[]


    for root, dirs, files in os.walk(directory):
        
        if len(dirs) > 0:
            listadirs = dirs

        if len(files) > 0:
            for value in files:
                listafiles.append(root.replace("/", "\\")+"\\"+value.replace("/","\\"))

            dic[listadirs[i]] = listafiles
            listafiles = []
            i+=1    
        
        for file in files:
            if file.endswith(ext):
                if filtro is not None:
                    if filtro.find(file.replace(".geojson","")) != -1 and len(file) == len(filtro):                   
                        ficheiro = os.path.join(root, file)
                        filtrolist.append(filtro)
                        lista.append(gpd.read_file(ficheiro))
            

def pprint():
    for k,v in dic.items():
        print(k+":")
        for value in v:
            print("       "+ value)


leitura()   
pprint()

ambiente:
       .\archives\ambiente\Amb_EcopontosSubterraneos.geojson
       .\archives\ambiente\Amb_Limpeza.geojson
       .\archives\ambiente\Ecopontos.geojson
       .\archives\ambiente\Jardins-Parques-urbanos.geojson
       .\archives\ambiente\Reciclagem.geojson
culturalazer:
       .\archives\culturalazer\Desporto_Instalacoes.geojson
       .\archives\culturalazer\Museus, bibliotecas e arquivos.geojson
economia:
       .\archives\economia\Centros Comerciais.geojson
       .\archives\economia\Feiras.geojson
       .\archives\economia\Mercados.geojson
educacao:
       .\archives\educacao\Escola Secundaria.geojson
       .\archives\educacao\escolas 1 ciclo.geojson
       .\archives\educacao\Escolas 1_Ciclo.geojson
saude:
       .\archives\saude\Farmacias.geojson
       .\archives\saude\Hospitais Publicos.geojson
seguranca:
       .\archives\seguranca\Administracao_Publica.geojson
transportes:
       .\archives\transportes\Estacoes Comboio.geojson
       .\archives\transportes\Estaco

In [169]:
#code with the objective os input severals directory
'''b=True
c=0


while b:
    
    a = input("Insira os diretorios: ")
    leitura(a)
    if a == "sair":
        b=False
'''
leitura('Feiras.geojson')
leitura('Escolas 1_Ciclo.geojson')
leitura('Hospitais Publicos.geojson')
leitura('Administracao_Publica.geojson')
leitura('Estacoes Fluviais.geojson')
leitura('Mercados.geojson')
leitura('Farmacias.geojso')
leitura('Hospitais Publicos.geojson')
leitura('Estacoes Metro.geojson')
leitura('Desporto_Instalacoes.geojson')





'''fig, ax=plt.subplots()
for a in lista:
    a.plot(ax=ax, zorder=c)
    c+=1

axis = df_places.plot(ax=ax, color = "white", edgecolor="black")


html_str = mpld3.fig_to_html(fig)
Html_file= open("index.html","w")
Html_file.write(html_str)
Html_file.close()'''

'fig, ax=plt.subplots()\nfor a in lista:\n    a.plot(ax=ax, zorder=c)\n    c+=1\n\naxis = df_places.plot(ax=ax, color = "white", edgecolor="black")\n\n\nhtml_str = mpld3.fig_to_html(fig)\nHtml_file= open("index.html","w")\nHtml_file.write(html_str)\nHtml_file.close()'

In [170]:
#creation of a dictionary with all the importance quantities for each file
amount = []
importance = {}

for a in range(len(filtrolist)):
    
    #b = float(input("Insira a importancia do filtro "+filtrolist[a]+":"))
    amount.append(5)
    importance[filtrolist[a]]=amount[a]

In [171]:
'''Choropleth with Points of Filter'''

geopoints = []
geopointsfiltered = []
geopointsforexplore = {}
count = []
total = []


#organizing the names
df_places.NOME=sorted(df_places.NOME)



for b in range(len(lista)):
    #append da geometry de filtro por filtro
    for idx, row in lista[b].iterrows():
        geopoints.append(row.geometry)
        
    for ele in geopoints:
        if ele != None:
            geopointsfiltered.append(ele)
            
    #FOR para analisar a intesecção dos pontos com as regiões 
    for a in geopointsfiltered:
        for idx1, row1 in df_places.iterrows():
                if a.intersects(row1.geometry) == True:
                    count.append(row1.NOME) #append em count da repetição das localidades


    #for para analisar a contagem dos pontos de cada regiao realizando um append a "total"
    for a in df_places.NOME:
        numb = count.count(a)
        total.append(numb) 

    #transformação para int         
    for a in range(len(total)):
        total[a] = float(total[a])

    
    fp[filtrolist[b]] = total
    
    geopointsforexplore[filtrolist[b]] = geopointsfiltered
    geopoints = []
    geopointsfiltered= []
    count = []
    total = []

In [172]:
sum=0
#organizing the names of Lisboa 
df_places.NOME=sorted(df_places.NOME)

#soma todos os pontos e dps divide pela quantidade total 
for a in range(len(filtrolist)):
    for b in range(0, len(fp[filtrolist[a]])):
        sum+=fp[filtrolist[a]][b]
    for b in range(0, len(fp[filtrolist[a]])): 
        fp[filtrolist[a]][b] = (fp[filtrolist[a]][b] / sum) * importance[filtrolist[a]]

    sum=0 



choropleth=[]
#soma dos pontos das mesmas regioes 
for a in range(len(fp[filtrolist[0]])):
    for b in range(0, len(filtrolist)):
        sum += fp[filtrolist[b]][a]
    
    #print(sum)
    choropleth.append(sum)
    sum=0
 

df_places["Censos 2021 População Lisboa_field_4"] = choropleth


In [175]:
color = []

for i in range(len(lista)):
    color.append('#%06X' % random.randint(0, 0xFFFFFF))

plot2 = df_places.explore(column = "Censos 2021 População Lisboa_field_4", 
                      cmap='YlOrRd', style_kwds=dict(color = "black"))


for a in range(len(lista)):
    axa=random.randint(0,len(color)-1)
    lista[a] = lista[a][lista[a].geometry != None]
    d = {'name': filtrolist[a].replace('.geojson', ''), 'geometry': lista[a].geometry}
    gdf = gpd.GeoDataFrame(d, crs="EPSG:4326")
    gdf.explore(m=plot2, 
                color = color.pop(axa),  
                style_kwds=dict(weight=9),
                popup=True)


plot2.save("explore.html")